# Aujourd'hui on se fait rouler par les mecs de l'ENS


https://challengedata.ens.fr/en/challenge/39/prediction_of_transaction_claims_status.html


Ici, c'est le notebook dédié au feature enginering - aka: le plus important de tous

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python

#load packages
import sys #access to system parameters https://docs.python.org/3/library/sys.html
print("Python version: {}". format(sys.version))

import pandas as pd #collection of functions for data processing and analysis modeled after R dataframes with SQL like features
print("pandas version: {}". format(pd.__version__))

import matplotlib #collection of functions for scientific and publication-ready visualization
print("matplotlib version: {}". format(matplotlib.__version__))

import numpy as np #foundational package for scientific computing
print("NumPy version: {}". format(np.__version__))

import scipy as sp #collection of functions for scientific computing and advance mathematics
print("SciPy version: {}". format(sp.__version__)) 

import IPython
from IPython import display #pretty printing of dataframes in Jupyter notebook
print("IPython version: {}". format(IPython.__version__)) 

import sklearn #collection of machine learning algorithms
print("scikit-learn version: {}". format(sklearn.__version__))

import seaborn as sns
print("seaborn version: {}". format(sns.__version__))

#misc libraries
import time
import random as rnd
import os, gc

#ignore warnings
import warnings
warnings.filterwarnings('ignore')
print('-'*25)

Python version: 3.6.3 |Anaconda custom (64-bit)| (default, Oct 15 2017, 03:27:45) [MSC v.1900 64 bit (AMD64)]
pandas version: 0.22.0
matplotlib version: 2.1.2
NumPy version: 1.12.1
SciPy version: 1.0.0
IPython version: 6.2.1
scikit-learn version: 0.19.1
seaborn version: 0.8.1
-------------------------


Moi j'ai ça:

Python version: 3.6.3 |Anaconda custom (64-bit)| (default, Oct 15 2017, 03:27:45) [MSC v.1900 64 bit (AMD64)]

pandas version: 0.22.0

matplotlib version: 2.1.2

NumPy version: 1.12.1

SciPy version: 1.0.0

IPython version: 6.2.1

scikit-learn version: 0.19.1

seaborn version: 0.8.1


In [2]:
# Run "! pip install jyquickhelper" dans une cellule si ca ne marche pas la commande suivante
from jyquickhelper import add_notebook_menu
add_notebook_menu()

In [3]:
# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

## Chargement des données

In [4]:
# Where to save the figures
PROJECT_ROOT_DIR = "."
DATA_PROCESSED = os.path.join(PROJECT_ROOT_DIR, "data_processed")

def load_data(file,data_path=DATA_PROCESSED, sep=','):
    csv_path = os.path.join(data_path, file)
    return pd.read_csv(csv_path, sep)

train_df = load_data(file = "train.csv", sep=';');
test_df = load_data(file = "test.csv", sep=';');


## Analyse des types de données

In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 22 columns):
SHIPPING_MODE           100000 non-null object
SHIPPING_PRICE          100000 non-null int64
WARRANTIES_FLG          100000 non-null bool
WARRANTIES_PRICE        100000 non-null int64
CARD_PAYMENT            100000 non-null int64
COUPON_PAYMENT          100000 non-null int64
RSP_PAYMENT             100000 non-null int64
WALLET_PAYMENT          100000 non-null int64
PRICECLUB_STATUS        100000 non-null int64
REGISTRATION_DATE       100000 non-null int64
PURCHASE_COUNT          100000 non-null int64
BUYER_BIRTHDAY_DATE     100000 non-null float64
BUYER_DEPARTMENT        100000 non-null int64
BUYING_DATE             100000 non-null int64
SELLER_SCORE_COUNT      100000 non-null int64
SELLER_SCORE_AVERAGE    100000 non-null float64
SELLER_COUNTRY          100000 non-null object
SELLER_DEPARTMENT       100000 non-null int64
PRODUCT_TYPE            100000 non-null object
PRODUCT_FA

## Plan d'action:

Idées de features engineering:
1. Taux de couverture de la garantie -> `WARRANTY_COV_RATE`. Ou faire la perte en cas de claim 
2. Classer les `SHIPPING_MODE` en fonction de leurs prix et/ou des services proposés
3. `SELLER_COUNTRY` en fonction de leurs PIB/habitants -> `SELLER_COUNTRY_PIB`
4. La densité de population par departement, pour montrer les grosses villes ou non (densité parce que 2 infos : nombre/superficie) -> `BUYER_DEPARTMENT_DENSITY` & `SELLER_DEPARTMENT_DENSITY`
5. Les PIB des départements -> `BUYER_DEPARTMENT_PIB` & `SELLER_DEPARTMENT_PIB`
6. Le niveau d'économie générale -> `CAC_POINTS` & `CAC_VAR`
7. A au moins une semaine de vacances scolaires dans le mois -> `IS_HOLIDAYS`
8. A des soldes dans le mois -> `IS_SALES`
9. Le niveau de température par mois -> `TEMPERATURE` (pas encore faite)
10. Le taux de chomage par départements -> `UNEMPLOYMENT` (pas urgent, certainement très corrélé avec le PIB)


Trouver un moyen de retrouver les clients via les données personelles 

Je vais utiliser que des `pandas.Series.map(arg, na_action=None)` pour s'assurer de ne pas laisser des trous dans la raquette.

A chaque fois j'utilise un `mapper` qui ressemble à `{valeur_à_remplacer_1: nouvelle_valeur_1, valeur_à_remplacer_2: nouvelle_valeur_2, etc...}`.

Aussi, je vais remplacer les valeurs manquante des départements par des valeurs extrèmes comme des `-9999` ou `-99` en fonction de l'intervalle de la variable

In [6]:
#train_df.info()
#print('_'*40)
#test_df.info()

### WARRANTY_COV_RATE

In [7]:
## ITEM_PRICE
print(train_df.ITEM_PRICE.sort_values().unique())
print(test_df.ITEM_PRICE.sort_values().unique())

[0 1 2 3 4 5 6]
[0 1 2 3 4 5 6]


In [8]:
## WARRANTIES_PRICE
print(train_df.WARRANTIES_PRICE.sort_values().unique())
print(test_df.WARRANTIES_PRICE.sort_values().unique())

[0 1 2 3 4 5]
[0 1 2 3 4 5]


Bon, comme il y a des 0 dans les Item Prices, on va devoir faire (WARRANTIES_PRICE)/(ITEM_PRICE+1)

In [9]:
train_df['WARRANTY_COV_RATE'] = (train_df.WARRANTIES_PRICE)/(train_df.ITEM_PRICE+1)
test_df['WARRANTY_COV_RATE'] = (test_df.WARRANTIES_PRICE)/(test_df.ITEM_PRICE+1)

In [10]:
print(train_df.WARRANTY_COV_RATE.sort_values().unique())
print(test_df.WARRANTY_COV_RATE.sort_values().unique())

[ 0.          0.2         0.25        0.33333333  0.4         0.5
  0.57142857  0.6         0.66666667  0.71428571  0.75        0.8
  0.83333333  1.          2.        ]
[ 0.          0.2         0.25        0.33333333  0.4         0.5
  0.57142857  0.6         0.66666667  0.71428571  0.75        0.8
  0.83333333  1.          2.        ]


### SELLER_COUNTRY_GINI

In [11]:
# A faire

### SELLER_COUNTRY_PIB

In [12]:
print(train_df.SELLER_COUNTRY.sort_values().unique())
print(test_df.SELLER_COUNTRY.sort_values().unique())

['ANDORRA' 'AUSTRIA' 'BELGIUM' 'BULGARIA' 'CANADA' 'CHINA' 'CYPRUS'
 'ESTONIA' 'FRANCE, METROPOLITAN' 'GERMANY' 'GIBRALTAR' 'GREECE' 'GUYANA'
 'HONG KONG' 'IRELAND' 'ISRAEL' 'ITALY' 'JAPAN' 'JERSEY' 'LATVIA'
 'LIECHTENSTEIN' 'LITHUANIA' 'LUXEMBOURG' 'MALAYSIA' 'MALTA' 'MARTINIQUE'
 'NETHERLANDS' 'POLAND' 'PORTUGAL' 'ROMANIA' 'SINGAPORE'
 'SLOVAKIA (Slovak Republic)' 'SLOVENIA' 'SPAIN' 'SWEDEN' 'SWITZERLAND'
 'UNITED KINGDOM' 'UNITED STATES' 'VATICAN CITY STATE (HOLY SEE)']
['ANDORRA' 'AUSTRIA' 'BELGIUM' 'BULGARIA' 'CANADA' 'CHINA' 'ESTONIA'
 'FRANCE, METROPOLITAN' 'GERMANY' 'GIBRALTAR' 'GUYANA' 'HONG KONG'
 'IRELAND' 'ISRAEL' 'ITALY' 'JAPAN' 'JERSEY' 'LATVIA' 'LIECHTENSTEIN'
 'LUXEMBOURG' 'MALAYSIA' 'MARTINIQUE' 'NETHERLANDS' 'POLAND' 'PORTUGAL'
 'ROMANIA' 'SINGAPORE' 'SLOVAKIA (Slovak Republic)' 'SLOVENIA' 'SPAIN'
 'SWEDEN' 'SWITZERLAND' 'UNITED KINGDOM' 'UNITED STATES'
 'VATICAN CITY STATE (HOLY SEE)']


In [13]:
mapper={ 'ANDORRA':40215, 'AUSTRIA':49038, 'BELGIUM':45537, 'BULGARIA':7328, 'CANADA':52037, 'CHINA':6958, 'CYPRUS':24866, 'ESTONIA':18852, 'FRANCE, METROPOLITAN':44098, 'GERMANY':44999, 'GIBRALTAR': 38200, 'GREECE':21857, 'GUYANA':3755, 'HONG KONG':37955, 'IRELAND':48607, 'ISRAEL':36926, 'ITALY':34714, 'JAPAN':38467, 'JERSEY':49500, 'LATVIA':15187, 'LIECHTENSTEIN':179478, 'LUXEMBOURG':112472, 'LITHUANIA':15648, 'MALAYSIA':10456, 'MALTA':22892, 'MARTINIQUE':10700, 'NETHERLANDS':50815, 'POLAND':13435, 'PORTUGAL':20995, 'ROMANIA':8873, 'SINGAPORE':55182, 'SLOVAKIA (Slovak Republic)':16499, 'SLOVENIA':23316, 'SPAIN':29150, 'SWEDEN':58014, 'SWITZERLAND':81276, 'UNITED KINGDOM':39371, 'UNITED STATES':53000, 'VATICAN CITY STATE (HOLY SEE)':21198 }

train_df['SELLER_COUNTRY_PIB'] = train_df.SELLER_COUNTRY.map(mapper)
test_df['SELLER_COUNTRY_PIB'] = test_df.SELLER_COUNTRY.map(mapper)

### SELLER_COUNTRY_DISTANCE

In [14]:
mapper={ 'ANDORRA':709, 'AUSTRIA':1037, 'BELGIUM':264, 'BULGARIA':1761, 'CANADA':5665, 'CHINA':8237, 'CYPRUS':2954, 'ESTONIA':1863, 'FRANCE, METROPOLITAN':0, 'GERMANY':878, 'GIBRALTAR': 1546, 'GREECE':2099, 'GUYANA':7074, 'HONG KONG':9645, 'IRELAND':782, 'ISRAEL':3282, 'ITALY':1107, 'JAPAN':9736, 'JERSEY':400, 'LATVIA':1707, 'LIECHTENSTEIN':568, 'LUXEMBOURG':288, 'LITHUANIA':1701, 'MALAYSIA':10429, 'MALTA':1749, 'MARTINIQUE':6860, 'NETHERLANDS':430, 'POLAND':1370, 'PORTUGAL':1453, 'ROMANIA':1875, 'SINGAPORE':10742, 'SLOVAKIA (Slovak Republic)':1092, 'SLOVENIA':967, 'SPAIN':1053, 'SWEDEN':1547, 'SWITZERLAND':81276, 'UNITED KINGDOM':344, 'UNITED STATES':5853, 'VATICAN CITY STATE (HOLY SEE)':1104 }

train_df['SELLER_COUNTRY_DISTANCE'] = train_df.SELLER_COUNTRY.map(mapper)
test_df['SELLER_COUNTRY_DISTANCE'] = test_df.SELLER_COUNTRY.map(mapper)

### BUYER_DEPARTMENT_DENSITY

In [15]:
mapper={0:-99,1:106,10:59,11:58,12:32,13:392,14:124,15:26,16:59,17:92,18:43,19:41,20:36,2:73,21:60,22:87,23:21,24:46,25:101,26:75,27:98,28:73,29:134,3:47,30:125,31:206,32:30,33:151,34:179,35:151,36:34,37:97,38:166,39:52,4:23,40:43,41:52,42:158,43:45,44:195,45:98,46:33,47:62,48:14,49:112,5:25,50:84,51:70,52:30,53:51,54:139,55:31,56:108,57:168,58:31,59:452,6:251,60:139,61:48,62:220,63:80,64:86,65:51,66:111,67:233,68:215,69:548,7:58,70:45,71:65,72:91,73:70,74:175,75:21154,76:200,77:231,78:621,79:62,8:54,80:93,81:66,82:66,83:172,84:157,85:97,86:62,87:68,88:64,89:46,9:31,90:235,91:695,92:9062,93:6573,94:5526,95:959,971:248,972:344,973:3,974:334,976:566}

train_df['BUYER_DEPARTMENT_DENSITY'] = train_df.BUYER_DEPARTMENT.map(mapper)
test_df['BUYER_DEPARTMENT_DENSITY'] = test_df.BUYER_DEPARTMENT.map(mapper)

### SELLER_DEPARTMENT_DENSITY

In [16]:
train_df['SELLER_DEPARTMENT_DENSITY'] = train_df.SELLER_DEPARTMENT.map(mapper)
test_df['SELLER_DEPARTMENT_DENSITY'] = test_df.SELLER_DEPARTMENT.map(mapper)

### BUYER_DEPARTMENT_PIB

In [17]:
mapper={0:-99999,1:22850,2:19908,3:21489,4:21135,5:25872,6:27723,7:19537,8:20263,9:19782,10:24089,11:20595,12:22191,13:27818,14:23724,15:19652,16:23736,17:20919,18:22272,19:22842,21:28139,22:21179,23:18475,24:18953,25:25700,26:25574,27:20986,28:21769,29:24094,20:23783,30:20589,31:29869,32:19791,33:27452,34:22999,35:27788,36:20667,37:24531,38:26355,39:22515,40:22749,41:22575,42:22160,43:19460,44:27310,45:28681,46:21101,47:20825,48:20889,49:22743,50:21767,51:29369,52:21388,53:23713,54:23540,55:20236,56:22283,57:22643,58:19769,59:23567,60:22656,61:20885,62:18841,63:25921,64:25518,65:22664,66:20393,67:26815,68:23927,69:34257,70:19070,71:22061,72:24283,73:29532,74:26935,75:75439,76:26583,77:23480,78:30507,79:24984,80:22513,81:20081,82:20773,83:22915,84:23280,85:23500,86:23190,87:23619,88:21830,89:22198,90:23982,91:26718,92:73277,93:27420,94:29250,95:25765,971:23000,972:23000,973:23000,974:23000,976:23000}

train_df['BUYER_DEPARTMENT_PIB'] = train_df.BUYER_DEPARTMENT.map(mapper)
test_df['BUYER_DEPARTMENT_PIB'] = test_df.BUYER_DEPARTMENT.map(mapper)

### SELLER_DEPARTMENT_PIB

In [18]:
train_df['SELLER_DEPARTMENT_PIB'] = train_df.SELLER_DEPARTMENT.map(mapper)
test_df['SELLER_DEPARTMENT_PIB'] = test_df.SELLER_DEPARTMENT.map(mapper)

### CAC_POINTS

In [19]:
mapper={ 1:4748, 2:4858, 3:5122, 4:5267, 5:5283, 6:5120, 7:5093, 8:5085, 9:5329, 10:5503, 11:5372, 12:5316 }

train_df['CAC_POINTS'] = train_df.BUYING_DATE.map(mapper)
test_df['CAC_POINTS'] = test_df.BUYING_DATE.map(mapper)

### CAC_VAR

In [20]:
mapper={1:-2, 2:2, 3:5, 4:3, 5:0, 6:-3, 7:-1, 8:0, 9:5, 10:3, 11:-2, 12:-1}

train_df['CAC_VAR'] = train_df.BUYING_DATE.map(mapper)
test_df['CAC_VAR'] = test_df.BUYING_DATE.map(mapper)

### IS_HOLIDAYS

In [21]:
mapper={1:1, 2:1, 3:0, 4:0, 5:0, 6:0, 7:1, 8:1, 9:0, 10:1, 11:0, 12:1}

train_df['IS_HOLIDAYS'] = train_df.BUYING_DATE.map(mapper)
test_df['IS_HOLIDAYS'] = test_df.BUYING_DATE.map(mapper)

### IS_SALES

In [22]:
mapper={1:1, 2:1, 3:0, 4:0, 5:0, 6:1, 7:1, 8:1, 9:0, 10:0, 11:0, 12:0}

train_df['IS_SALES'] = train_df.BUYING_DATE.map(mapper)
test_df['IS_SALES'] = test_df.BUYING_DATE.map(mapper)

In [23]:
train_df.info()
print('_'*40)
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 33 columns):
SHIPPING_MODE                100000 non-null object
SHIPPING_PRICE               100000 non-null int64
WARRANTIES_FLG               100000 non-null bool
WARRANTIES_PRICE             100000 non-null int64
CARD_PAYMENT                 100000 non-null int64
COUPON_PAYMENT               100000 non-null int64
RSP_PAYMENT                  100000 non-null int64
WALLET_PAYMENT               100000 non-null int64
PRICECLUB_STATUS             100000 non-null int64
REGISTRATION_DATE            100000 non-null int64
PURCHASE_COUNT               100000 non-null int64
BUYER_BIRTHDAY_DATE          100000 non-null float64
BUYER_DEPARTMENT             100000 non-null int64
BUYING_DATE                  100000 non-null int64
SELLER_SCORE_COUNT           100000 non-null int64
SELLER_SCORE_AVERAGE         100000 non-null float64
SELLER_COUNTRY               100000 non-null object
SELLER_DEPARTMENT 

Actuellement, il nous reste encore quatre variables catégorielles:
1. `SHIPPING_MODE`
2. `SELLER_COUNTRY`
3. `PRODUCT_TYPE`
4. `PRODUCT_FAMILY`

Mais bon, `SELLER_COUNTRY` a maintenant plusieurs représentations numériques, donc en vrai, il n'en reste que 3.

On va voir ce qu'on peut faire avec `PRODUCT_TYPE` dans le  `Notebook 3 Stat des Sellers et clients`

Mais quel teasing!

Et on oublie pas d'écrire les résultats de nos transformations.

In [24]:
filename_train = DATA_PROCESSED+"/train.csv"
filename_test = DATA_PROCESSED+"/test.csv"

try:
    os.remove(filename_train)
    os.remove(filename_test)
except:
    pass;

train_df.to_csv(filename_train, index=False, sep=";")
test_df.to_csv(filename_test, index=False, sep=";")